In [1]:
%reload_ext autoreload
%autoreload 2
%reload_ext notexbook
%texify

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import time
import sys
sys.setrecursionlimit(10000)
%matplotlib qt5

In [3]:
import cellbgnet
import cellbgnet.utils
from cellbgnet.datasets import DataSimulator
from cellbgnet.utils.hardware import cpu, gpu
from cellbgnet.model import CellBGModel
from cellbgnet.utils.plot_funcs import plot_od, plot_train_record

In [4]:
param_file = '../../cellbgnet/utils/reference_files/reference_beads.yaml'
param = cellbgnet.utils.param_io.ParamHandling().load_params(param_file)

In [5]:
psf_params = param.PSF.to_dict()
simulation_params = param.Simulation.to_dict()
hardware_params = param.Hardware.to_dict()
eval_params = param.Evaluation.to_dict()
train_size = simulation_params['train_size']

In [6]:
model = cellbgnet.model.CellBGModel(param)

training sliding windows on camera chip:
Area num: 0, field_xy: [0, 127, 0, 127]
Area num: 1, field_xy: [114, 241, 0, 127]
Area num: 2, field_xy: [228, 355, 0, 127]
Area num: 3, field_xy: [342, 469, 0, 127]
Area num: 4, field_xy: [456, 583, 0, 127]
Area num: 5, field_xy: [570, 697, 0, 127]
Area num: 6, field_xy: [684, 811, 0, 127]
Area num: 7, field_xy: [798, 925, 0, 127]
Area num: 8, field_xy: [912, 1039, 0, 127]
Area num: 9, field_xy: [1026, 1153, 0, 127]
Area num: 10, field_xy: [1140, 1267, 0, 127]
Area num: 11, field_xy: [1174, 1301, 0, 127]
Area num: 12, field_xy: [0, 127, 114, 241]
Area num: 13, field_xy: [114, 241, 114, 241]
Area num: 14, field_xy: [228, 355, 114, 241]
Area num: 15, field_xy: [342, 469, 114, 241]
Area num: 16, field_xy: [456, 583, 114, 241]
Area num: 17, field_xy: [570, 697, 114, 241]
Area num: 18, field_xy: [684, 811, 114, 241]
Area num: 19, field_xy: [798, 925, 114, 241]
Area num: 20, field_xy: [912, 1039, 114, 241]
Area num: 21, field_xy: [1026, 1153, 114, 24

In [7]:
model.init_recorder()
model.init_eval_data()

Eval image generation: 100%|████████████████████| 30/30 [00:02<00:00, 11.80it/s]


eval images shape: (30, 1041, 1302) contain 2928 molecules,


In [8]:
model.evaluation_params['eval_imgs'].shape

(30, 1041, 1302)

In [9]:
model.simulation_params

{'train_size': 128,
 'train_type': 'uniform',
 'psf_intensity_normalization': True,
 'density': 0.003125,
 'non_cell_density': 10,
 'margin_empty': 0.05,
 'z_prior': [-1, 1],
 'survival_prob': None,
 'min_photon': 0.05,
 'camera': 'sCMOS',
 'camera_chip_size': [1041, 1302],
 'perlin_noise': False,
 'perlin_noise_factor': 0.2,
 'perlin_noise_res': 64,
 'baseline': 103.0,
 'bg_values': 600.0,
 'bg_width': 150,
 'e_per_adu': 0.38,
 'qe': 0.95,
 'spurious_c': 0.0,
 'sig_read': '/mnt/sda1/SMLAT/data/real_data/dark_images/RN_e_venus_upgraded.tiff',
 'robust_training': False,
 'cell_masks_dir': '/mnt/sda1/SMLAT/data/real_data/chromosome_dots/EXP-23-CA3045/pool_3045_ter_pos01/phase_venus_mask/',
 'cell_masks_filetype': '.png',
 'edt_noise_map_path': '/mnt/sda1/SMLAT/data/real_data/chromosome_dots/EXP-23-CA3045/edt_noise.pkl',
 'min_edt': 0,
 'max_edt': 6,
 'augment': True}

In [10]:
start_time = time.time()
model.fit(batch_size=32, max_iters=10000, print_output=True, print_freq=100)
duration = time.time() - start_time

Started training .... 
 || Cost: 21299.744 || Time Upd.: 677.7 ms  || BatchNr.: 100
 || Cost: 809.255 || Time Upd.: 651.8 ms  || BatchNr.: 200
 || Cost: 417.594 || Time Upd.: 653.1 ms  || BatchNr.: 300
 || Cost: 272.683 || Time Upd.: 656.1 ms  || BatchNr.: 400
 || Cost: 214.927 || Time Upd.: 651.5 ms  || BatchNr.: 500
 || Cost: 186.476 || Time Upd.: 647.4 ms  || BatchNr.: 600
 || Cost: 155.783 || Time Upd.: 646.3 ms  || BatchNr.: 700
 || Cost: 126.565 || Time Upd.: 668.7 ms  || BatchNr.: 800
 || Cost: 144.436 || Time Upd.: 666.6 ms  || BatchNr.: 900
 || Cost: 119.497 || Time Upd.: 670.2 ms  || BatchNr.: 1000
processing area:99/99, input field_xy:[1258 1301 1001 1040], use_coordconv:True, retain locs in area:[1278, 1301, 1021, 1040]

evaluation on 30 images, contain ground truth: 2928FOV: x=[0, 84630] y=[0, 67665]
after FOV and border segmentation,truth: 2891 ,preds: 2863
JoR: 8.508 || Eff_3d: 71.858 || Jaccard: 0.988 || Factor: 100.847 || RMSE_lat: 11.616 || RMSE_ax: 89.186 || RMSE_x: 

processing area:99/99, input field_xy:[1258 1301 1001 1040], use_coordconv:True, retain locs in area:[1278, 1301, 1021, 1040]

evaluation on 30 images, contain ground truth: 2928FOV: x=[0, 84630] y=[0, 67665]
after FOV and border segmentation,truth: 2891 ,preds: 2858
JoR: 10.717 || Eff_3d: 84.833 || Jaccard: 0.987 || Factor: 95.571 || RMSE_lat: 9.212 || RMSE_ax: 41.990 || RMSE_x: 6.473 || RMSE_y: 6.554 || Cost: 29.855 || Recall: 0.988 || Precision: 0.999 || BatchNr.: 2200 || Time Upd.: 703.1 ms 
 || Count Cost: 106.9340 || Localization Cost: -19.6301 || Bg Cost: 0.1661 || P_locs Cost: 59.2360saving this model, eff_3d, rmse_lat, rmse_ax and BatchNr are : 84.833 || 9.212 || 41.990 || 2200


processing area:99/99, input field_xy:[1258 1301 1001 1040], use_coordconv:True, retain locs in area:[1278, 1301, 1021, 1040]

evaluation on 30 images, contain ground truth: 2928FOV: x=[0, 84630] y=[0, 67665]
after FOV and border segmentation,truth: 2891 ,preds: 2927
JoR: 8.436 || Eff_3d: 77.717 || Ja

processing area:99/99, input field_xy:[1258 1301 1001 1040], use_coordconv:True, retain locs in area:[1278, 1301, 1021, 1040]

evaluation on 30 images, contain ground truth: 2928FOV: x=[0, 84630] y=[0, 67665]
after FOV and border segmentation,truth: 2891 ,preds: 2849
JoR: 14.984 || Eff_3d: 88.100 || Jaccard: 0.983 || Factor: 96.893 || RMSE_lat: 6.559 || RMSE_ax: 33.860 || RMSE_x: 4.315 || RMSE_y: 4.939 || Cost: -20.549 || Recall: 0.984 || Precision: 0.999 || BatchNr.: 3400 || Time Upd.: 663.2 ms 
 || Count Cost: 102.1957 || Localization Cost: -61.0355 || Bg Cost: 0.0942 || P_locs Cost: 50.0536saving this model, eff_3d, rmse_lat, rmse_ax and BatchNr are : 88.100 || 6.559 || 33.860 || 3400


processing area:99/99, input field_xy:[1258 1301 1001 1040], use_coordconv:True, retain locs in area:[1278, 1301, 1021, 1040]

evaluation on 30 images, contain ground truth: 2928FOV: x=[0, 84630] y=[0, 67665]
after FOV and border segmentation,truth: 2891 ,preds: 2855
JoR: 12.081 || Eff_3d: 88.826 || 

processing area:99/99, input field_xy:[1258 1301 1001 1040], use_coordconv:True, retain locs in area:[1278, 1301, 1021, 1040]

evaluation on 30 images, contain ground truth: 2928FOV: x=[0, 84630] y=[0, 67665]
after FOV and border segmentation,truth: 2891 ,preds: 2876
JoR: 14.555 || Eff_3d: 90.346 || Jaccard: 0.986 || Factor: 108.696 || RMSE_lat: 6.773 || RMSE_ax: 24.618 || RMSE_x: 4.379 || RMSE_y: 5.167 || Cost: -38.323 || Recall: 0.990 || Precision: 0.995 || BatchNr.: 4600 || Time Upd.: 675.7 ms 
 || Count Cost: 100.1471 || Localization Cost: -77.2661 || Bg Cost: 0.0946 || P_locs Cost: 45.3924saving this model, eff_3d, rmse_lat, rmse_ax and BatchNr are : 90.346 || 6.773 || 24.618 || 4600


processing area:99/99, input field_xy:[1258 1301 1001 1040], use_coordconv:True, retain locs in area:[1278, 1301, 1021, 1040]

evaluation on 30 images, contain ground truth: 2928FOV: x=[0, 84630] y=[0, 67665]
after FOV and border segmentation,truth: 2891 ,preds: 2900
JoR: 14.310 || Eff_3d: 88.070 ||

processing area:99/99, input field_xy:[1258 1301 1001 1040], use_coordconv:True, retain locs in area:[1278, 1301, 1021, 1040]

evaluation on 30 images, contain ground truth: 2928FOV: x=[0, 84630] y=[0, 67665]
after FOV and border segmentation,truth: 2891 ,preds: 2909
JoR: 16.628 || Eff_3d: 91.254 || Jaccard: 0.985 || Factor: 134.684 || RMSE_lat: 5.923 || RMSE_ax: 22.560 || RMSE_x: 4.171 || RMSE_y: 4.206 || Cost: -70.618 || Recall: 0.996 || Precision: 0.989 || BatchNr.: 5800 || Time Upd.: 646.6 ms 
 || Count Cost: 103.9296 || Localization Cost: -109.0586 || Bg Cost: 0.0942 || P_locs Cost: 40.4675saving this model, eff_3d, rmse_lat, rmse_ax and BatchNr are : 91.254 || 5.923 || 22.560 || 5800


processing area:99/99, input field_xy:[1258 1301 1001 1040], use_coordconv:True, retain locs in area:[1278, 1301, 1021, 1040]

evaluation on 30 images, contain ground truth: 2928FOV: x=[0, 84630] y=[0, 67665]
after FOV and border segmentation,truth: 2891 ,preds: 2923
JoR: 13.849 || Eff_3d: 90.594 |

processing area:99/99, input field_xy:[1258 1301 1001 1040], use_coordconv:True, retain locs in area:[1278, 1301, 1021, 1040]

evaluation on 30 images, contain ground truth: 2928FOV: x=[0, 84630] y=[0, 67665]
after FOV and border segmentation,truth: 2891 ,preds: 2910
JoR: 16.183 || Eff_3d: 91.886 || Jaccard: 0.989 || Factor: 141.943 || RMSE_lat: 6.109 || RMSE_ax: 19.902 || RMSE_x: 4.119 || RMSE_y: 4.512 || Cost: -76.913 || Recall: 0.998 || Precision: 0.991 || BatchNr.: 7000 || Time Upd.: 648.9 ms 
 || Count Cost: 96.1565 || Localization Cost: -108.1234 || Bg Cost: 0.0290 || P_locs Cost: 38.6460saving this model, eff_3d, rmse_lat, rmse_ax and BatchNr are : 91.886 || 6.109 || 19.902 || 7000


processing area:99/99, input field_xy:[1258 1301 1001 1040], use_coordconv:True, retain locs in area:[1278, 1301, 1021, 1040]

evaluation on 30 images, contain ground truth: 2928FOV: x=[0, 84630] y=[0, 67665]
after FOV and border segmentation,truth: 2891 ,preds: 2922
JoR: 16.172 || Eff_3d: 90.624 ||

processing area:99/99, input field_xy:[1258 1301 1001 1040], use_coordconv:True, retain locs in area:[1278, 1301, 1021, 1040]

evaluation on 30 images, contain ground truth: 2928FOV: x=[0, 84630] y=[0, 67665]
after FOV and border segmentation,truth: 2891 ,preds: 2916
JoR: 12.913 || Eff_3d: 91.751 || Jaccard: 0.989 || Factor: 113.719 || RMSE_lat: 7.657 || RMSE_ax: 17.369 || RMSE_x: 6.218 || RMSE_y: 4.468 || Cost: -85.933 || Recall: 0.999 || Precision: 0.990 || BatchNr.: 8200 || Time Upd.: 652.9 ms 
 || Count Cost: 94.6725 || Localization Cost: -119.3885 || Bg Cost: 0.0682 || P_locs Cost: 36.2564saving this model, eff_3d, rmse_lat, rmse_ax and BatchNr are : 91.751 || 7.657 || 17.369 || 8200


processing area:99/99, input field_xy:[1258 1301 1001 1040], use_coordconv:True, retain locs in area:[1278, 1301, 1021, 1040]

evaluation on 30 images, contain ground truth: 2928FOV: x=[0, 84630] y=[0, 67665]
after FOV and border segmentation,truth: 2891 ,preds: 2878
JoR: 15.704 || Eff_3d: 92.721 ||

processing area:99/99, input field_xy:[1258 1301 1001 1040], use_coordconv:True, retain locs in area:[1278, 1301, 1021, 1040]

evaluation on 30 images, contain ground truth: 2928FOV: x=[0, 84630] y=[0, 67665]
after FOV and border segmentation,truth: 2891 ,preds: 2934
JoR: 17.128 || Eff_3d: 93.370 || Jaccard: 0.985 || Factor: 196.170 || RMSE_lat: 5.749 || RMSE_ax: 14.295 || RMSE_x: 4.151 || RMSE_y: 3.977 || Cost: -100.164 || Recall: 1.000 || Precision: 0.985 || BatchNr.: 9400 || Time Upd.: 710.5 ms 
 || Count Cost: 95.4994 || Localization Cost: -131.0242 || Bg Cost: 0.0405 || P_locs Cost: 34.0884saving this model, eff_3d, rmse_lat, rmse_ax and BatchNr are : 93.370 || 5.749 || 14.295 || 9400


processing area:99/99, input field_xy:[1258 1301 1001 1040], use_coordconv:True, retain locs in area:[1278, 1301, 1021, 1040]

evaluation on 30 images, contain ground truth: 2928FOV: x=[0, 84630] y=[0, 67665]
after FOV and border segmentation,truth: 2891 ,preds: 2905
JoR: 14.253 || Eff_3d: 91.165 |

In [12]:
plot_train_record(model)